## https://github.com/TomAugspurger/effective-pandas/tree/updates
    

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Want to guess what the most popular pandas-related question on StackOverflow is about? http://stackoverflow.com/questions/14262433/large-data-work-flows-using-pandas

The strength of pandas really is in medium-data analytics, which can roughly be described as "datasets that fit in memory, comfortable".
Depending on your data needs (and ability to buy time on a big [EC2 instance](https://aws.amazon.com/ec2/instance-types/) with, say, 244 GiB of RAM), this section may not apply to you.

Pandas is not meant for "Big Data", but then again you probably don't have big data.

# Chunking and Iteration

The first potential operation for handling larger-than-memory data is chunking or batching your data, and iterating over each batch.
This immediately rules out algorithms that require the full dataset to be in memory at once, but with a bit of cleverness you can work around that limitation for many problems.

In [3]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

import matplotlib.pyplot as plt
import seaborn as sns

ImportError: cannot import name 'partial_by_order'

In [ ]:
pd.Timestamp('2014-01-01').strftime("%Y%m")

In [ ]:
from distributed import Executor

executor = Executor('127.0.0.1:8786')

In [ ]:
from distributed.diagnostics import progress

In [ ]:
progress?

In [ ]:
import os
import requests

In [ ]:
def download_month(month):
    os.makedirs('comext', exist_ok=True)
    base = ("http://ec.europa.eu/eurostat/estat-navtree-portlet-prod/"
            "BulkDownloadListing?sort=1&"
            "downfile=comext%2F2015S1%2Fdata%2Fnc{:%Y%m}.7z")
    r = requests.get(base.format(month), stream=True)
    filename = 'comext/{:%Y-%m}.tsv.7z'.format(month)
    with open(filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    return filename

    

In [ ]:
dates = pd.date_range(start='2012-01-01', end='2014-12-01', freq='m')
futures = executor.map(download_month, dates)
progress(*futures)

In [ ]:
!rename -S .gz .7z comext/*.gz

In [ ]:
rm nc201401.dat

In [ ]:
!7z x -ocomext comext/*.7z

In [ ]:
df = pd.read_csv('comext/nc201401.dat', dtype={'DECLARANT': 'object'})

In [ ]:
df.head()

In [ ]:
import dask.dataframe as dd

In [ ]:
import zipfile

In [ ]:
zf = zipfile.ZipFile('ml-latest.zip')

zf.extractall()

In [ ]:
ls ml-latest/

In [ ]:
df = pd.read_csv('ml-latest/ratings.csv')
df['timestamp'] = pd.to_datetime(df.timestamp, unit='s')

In [ ]:
ratings = dd.from_pandas(df, npartitions=100)

In [ ]:
ratings.head()

In [ ]:
s = df.head(1000000)

In [ ]:
s2 = dd.from_pandas(s, npartitions=20)

In [ ]:
def sessionize(ts):
    return (ts.sort_values().diff() >= pd.Timedelta(1, unit='h')).fillna(True).cumsum()

In [ ]:
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler

In [ ]:
with Profiler() as prof, ResourceProfiler() as rprof:
    out = ratings.groupby('userId').timestamp.apply(sessionize, columns='timstamp').compute()

In [ ]:
prof.visualize()

In [ ]:
rprof.visualize()

In [ ]:
%%time
s.groupby('userId').timestamp.apply(sessionize)

In [ ]:
%%time
s2.groupby(level=0).timestamp.apply(sessionize)

In [ ]:
df.groupby(['userId']).timestamp.apply(sessionize)

In [ ]:
ratings.groupby('userId').rating.apply(np.mean)

In [ ]:
df

In [ ]:
# %load ml-latest/README.txt
Summary
=======

This dataset (ml-latest) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 22884377 ratings and 586994 tag applications across 34208 movies. These data were created by 247753 users between January 09, 1995 and January 29, 2016. This dataset was generated on January 29, 2016.

Users were selected at random for inclusion. All selected users had rated at least 1 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in four files, `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLens data sets are publicly available for download at <http://grouplens.org/datasets/>.


Usage License
=============

Neither the University of Minnesota nor any of the researchers involved can guarantee the correctness of the data, its suitability for any particular purpose, or the validity of results based on the use of the data set. The data set may be used for any research purposes under the following conditions:

* The user may not state or imply any endorsement from the University of Minnesota or the GroupLens Research Group.
* The user must acknowledge the use of the data set in publications resulting from the use of the data set (see below for citation information).
* The user may not redistribute the data without separate permission.
* The user may not use this information for any commercial or revenue-bearing purposes without first obtaining permission from a faculty member of the GroupLens Research Project at the University of Minnesota.
* The executable software scripts are provided "as is" without warranty of any kind, either expressed or implied, including, but not limited to, the implied warranties of merchantability and fitness for a particular purpose. The entire risk as to the quality and performance of them is with you. Should the program prove defective, you assume the cost of all necessary servicing, repair or correction.

In no event shall the University of Minnesota, its affiliates or employees be liable to you for any damages arising out of the use or inability to use these programs (including but not limited to loss of data or data being rendered inaccurate).

If you have any further questions or comments, please email <grouplens-info@cs.umn.edu>


Citation
========

To acknowledge use of the dataset in publications, please cite the following paper:

> F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=<http://dx.doi.org/10.1145/2827872>


Further Information About GroupLens
===================================

GroupLens is a research group in the Department of Computer Science and Engineering at the University of Minnesota. Since its inception in 1992, GroupLens's research projects have explored a variety of fields including:

* recommender systems
* online communities
* mobile and ubiquitious technologies
* digital libraries
* local geographic information systems

GroupLens Research operates a movie recommender based on collaborative filtering, MovieLens, which is the source of these data. We encourage you to visit <http://movielens.org> to try it out! If you have exciting ideas for experimental work to conduct on MovieLens, send us an email at <grouplens-info@cs.umn.edu> - we are always interested in working with external collaborators.


Content and Use of Files
========================

Formatting and Encoding
-----------------------

The dataset files are written as [comma-separated values](http://en.wikipedia.org/wiki/Comma-separated_values) files with a single header row. Columns that contain commas (`,`) are escaped using double-quotes (`"`). These files are encoded as UTF-8. If accented characters in movie titles or tag values (e.g. Misérables, Les (1995)) display incorrectly, make sure that any program reading the data, such as a text editor, terminal, or script, is configured for UTF-8.

User Ids
--------

MovieLens users were selected at random for inclusion. Their ids have been anonymized. User ids are consistent between `ratings.csv` and `tags.csv` (i.e., the same id refers to the same user across the two files).

Movie Ids
---------

Only movies with at least one rating or tag are included in the dataset. These movie ids are consistent with those used on the MovieLens web site (e.g., id `1` corresponds to the URL <https://movielens.org/movies/1>). Movie ids are consistent between `ratings.csv`, `tags.csv`, `movies.csv`, and `links.csv` (i.e., the same id refers to the same movie across these four data files).


Ratings Data File Structure (ratings.csv)
-----------------------------------------

All ratings are contained in the file `ratings.csv`. Each line of this file after the header row represents one rating of one movie by one user, and has the following format:

    userId,movieId,rating,timestamp

The lines within this file are ordered first by userId, then, within user, by movieId.

Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).

Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.

Tags Data File Structure (tags.csv)
-----------------------------------

All tags are contained in the file `tags.csv`. Each line of this file after the header row represents one tag applied to one movie by one user, and has the following format:

    userId,movieId,tag,timestamp

The lines within this file are ordered first by userId, then, within user, by movieId.

Tags are user-generated metadata about movies. Each tag is typically a single word or short phrase. The meaning, value, and purpose of a particular tag is determined by each user.

Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.

Movies Data File Structure (movies.csv)
---------------------------------------

Movie information is contained in the file `movies.csv`. Each line of this file after the header row represents one movie, and has the following format:

    movieId,title,genres

Movie titles are entered manually or imported from <https://www.themoviedb.org/>, and include the year of release in parentheses. Errors and inconsistencies may exist in these titles.

Genres are a pipe-separated list, and are selected from the following:

* Action
* Adventure
* Animation
* Children's
* Comedy
* Crime
* Documentary
* Drama
* Fantasy
* Film-Noir
* Horror
* Musical
* Mystery
* Romance
* Sci-Fi
* Thriller
* War
* Western
* (no genres listed)

Links Data File Structure (links.csv)
---------------------------------------

Identifiers that can be used to link to other sources of movie data are contained in the file `links.csv`. Each line of this file after the header row represents one movie, and has the following format:

    movieId,imdbId,tmdbId

movieId is an identifier for movies used by <https://movielens.org>. E.g., the movie Toy Story has the link <https://movielens.org/movies/1>.

imdbId is an identifier for movies used by <http://www.imdb.com>. E.g., the movie Toy Story has the link <http://www.imdb.com/title/tt0114709/>.

tmdbId is an identifier for movies used by <https://www.themoviedb.org>. E.g., the movie Toy Story has the link <https://www.themoviedb.org/movie/862>.

Use of the resources listed above is subject to the terms of each provider.

Cross-Validation
----------------

Prior versions of the MovieLens dataset included either pre-computed cross-folds or scripts to perform this computation. We no longer bundle either of these features with the dataset, since most modern toolkits provide this as a built-in feature. If you wish to learn about standard approaches to cross-fold computation in the context of recommender systems evaluation, see [LensKit](http://lenskit.org) for tools, documentation, and open-source code examples.
